In [9]:
# import json
#
# # Load the OpenAPI JSON file
# with open("plaid-openapi.json", "r") as file:
#     openapi_data = json.load(file)
#
# def get_properties_from_ref(ref, components):
#     """
#     Given a $ref string, extract the actual schema it points to.
#     """
#     # Remove the '#/components/schemas/' prefix to get the actual schema name
#     schema_name = ref.replace("#/components/schemas/", "")
#     return components["schemas"][schema_name]
#
#
# def extract_properties(schema, components, prefix=""):
#     """
#     Recursively extract properties from a given schema.
#     """
#     properties = []
#
#     # If the schema is a reference, dereference it
#     if "$ref" in schema:
#         schema = get_properties_from_ref(schema["$ref"], components)
#
#     # If the schema has "allOf", merge the schemas
#     if "allOf" in schema:
#         merged_schema = {}
#         for subschema in schema["allOf"]:
#             if "$ref" in subschema:
#                 subschema = get_properties_from_ref(subschema["$ref"], components)
#             merged_schema.update(subschema)
#         schema = merged_schema
#
#     # If the schema is of type object and has properties, iterate over them
#     if schema.get("type") == "object" and "properties" in schema:
#         for prop_name, prop_schema in schema["properties"].items():
#             full_name = f"{prefix}.{prop_name}" if prefix else prop_name
#             if "$ref" in prop_schema:
#                 properties += extract_properties(prop_schema, components, full_name)
#             elif prop_schema.get("type") == "object":
#                 properties += extract_properties(prop_schema, components, full_name)
#             elif prop_schema.get("type") == "array" and "items" in prop_schema:
#                 properties += extract_properties(prop_schema["items"], components, full_name)
#             else:
#                 prop_details = {
#                     "property_name": full_name,
#                     "property_type": prop_schema.get("type", "N/A"),
#                     "property_description": prop_schema.get("description", "N/A"),
#                     "enum": prop_schema.get("enum", []),
#                     "nullable": prop_schema.get("nullable", False)
#                 }
#                 properties.append(prop_details)
#     elif schema.get("type") == "array" and "items" in schema:
#         # If the schema is of type array, extract properties from its items
#         properties += extract_properties(schema["items"], components, prefix)
#
#     return properties
#
#
# # Extract properties for the "transactions/get" endpoint's response schema
# endpoint_path = "/transactions/get"
# response_schema = openapi_data["paths"][endpoint_path]["post"]["responses"]["200"]["content"]["application/json"]["schema"]
# properties_list = extract_properties(response_schema, openapi_data["components"])
#
# properties_list

[{'property_name': 'accounts.account_id',
  'property_type': 'string',
  'property_description': "Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.\n\nThe `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.\n\nIf an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.\n\nLike all Plaid identifiers, the `account_id` is case sensitive.",
  'enum': [],
  'nullable': False},
 {'property_name': 'accounts.balances.available',
  'proper

In [16]:
# import json
#
# # Load the OpenAPI JSON file
# with open("plaid-openapi.json", "r") as file:
#     openapi_data = json.load(file)
#
# def get_properties_from_ref(ref, components):
#     """
#     Given a $ref string, extract the actual schema it points to.
#     """
#     # Remove the '#/components/schemas/' prefix to get the actual schema name
#     schema_name = ref.replace("#/components/schemas/", "")
#     schema = components["schemas"][schema_name]
#
#     # If the schema has "allOf", merge the schemas
#     if "allOf" in schema:
#         merged_schema = {}
#         for subschema in schema["allOf"]:
#             if "$ref" in subschema:
#                 subschema = get_properties_from_ref(subschema["$ref"], components)
#             merged_schema.update(subschema)
#         schema = merged_schema
#
#     return schema
#
#
# def extract_properties(schema, components, prefix=""):
#     """
#     Recursively extract properties from a given schema.
#     """
#     properties = []
#
#     # If the schema is a reference, dereference it
#     if "$ref" in schema:
#         schema = get_properties_from_ref(schema["$ref"], components)
#
#     # If the schema has "allOf", iterate over its schemas and extract properties
#     if "allOf" in schema:
#         for subschema in schema["allOf"]:
#             properties += extract_properties(subschema, components, prefix)
#
#     # If the schema is of type object and has properties, iterate over them
#     if schema.get("type") == "object" and "properties" in schema:
#         for prop_name, prop_schema in schema["properties"].items():
#             full_name = f"{prefix}.{prop_name}" if prefix else prop_name
#             properties += extract_properties(prop_schema, components, full_name)
#
#     elif schema.get("type") == "array" and "items" in schema:
#         # If the schema is of type array, extract properties from its items
#         full_name = prefix
#         properties += extract_properties(schema["items"], components, full_name)
#
#     else:
#         prop_details = {
#             "property_name": prefix,
#             "property_type": schema.get("type", "N/A"),
#             "property_description": schema.get("description", "N/A"),
#             "enum": schema.get("enum", []),
#             "nullable": schema.get("nullable", False)
#         }
#         properties.append(prop_details)
#
#     return properties
#
# endpoint_path = "/transactions/get"
# response_schema = openapi_data["paths"][endpoint_path]["post"]["responses"]["200"]["content"]["application/json"]["schema"]
# properties_list = extract_properties(response_schema, openapi_data["components"])
#
# properties_list

[{'property_name': 'accounts.account_id',
  'property_type': 'string',
  'property_description': "Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.\n\nThe `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.\n\nIf an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.\n\nLike all Plaid identifiers, the `account_id` is case sensitive.",
  'enum': [],
  'nullable': False},
 {'property_name': 'accounts.balances.available',
  'proper

In [44]:
import json

# Load the OpenAPI JSON file
with open("plaid-openapi.json", "r") as file:
    openapi_data = json.load(file)

def get_properties_from_ref(ref, components):
    """
    Given a $ref string, extract the actual schema it points to.
    """
    schema_name = ref.replace("#/components/schemas/", "")
    return components["schemas"][schema_name]


def merge_allOf_schemas(allOf, components):
    """
    Merge schemas under "allOf" into a single schema.
    """
    merged_schema = {"properties": {}}
    for subschema in allOf:
        if "$ref" in subschema:
            subschema = get_properties_from_ref(subschema["$ref"], components)
        if "properties" in subschema:
            merged_schema["properties"].update(subschema["properties"])
    return merged_schema


def extract_properties_from_allOf(allOf, components, prefix=""):
    """
    Extract properties from a given "allOf" list.
    """
    properties = []
    for subschema in allOf:
        properties += extract_properties(subschema, components, prefix)
    return properties


def extract_properties(schema, components, prefix=""):
    """
    Recursively extract properties from a given schema.
    """
    properties = []

    # If the schema is a reference, dereference it
    if "$ref" in schema:
        schema = get_properties_from_ref(schema["$ref"], components)

    # If the schema has "allOf", extract properties from each of the schemas within
    if "allOf" in schema:
        properties += extract_properties_from_allOf(schema["allOf"], components, prefix)

    # If the schema is of type object and has properties, iterate over them
    if schema.get("type") == "object" and "properties" in schema:
        for prop_name, prop_schema in schema["properties"].items():
            full_name = f"{prefix}.{prop_name}" if prefix else prop_name
            properties += extract_properties(prop_schema, components, full_name)
    elif schema.get("type") == "array" and "items" in schema:
        properties += extract_properties(schema["items"], components, prefix)
    else:
        prop_details = {
            "property_name": prefix,
            "property_type": schema.get("type", "N/A"),
            "property_description": schema.get("description", "N/A"),
            "enum": schema.get("enum", []),
            "nullable": schema.get("nullable", False)
        }
        properties.append(prop_details)

    return properties


endpoint_path = "/transactions/get"
response_schema = openapi_data["paths"][endpoint_path]["post"]["responses"]["200"]["content"]["application/json"]["schema"]
properties_list = extract_properties(response_schema, openapi_data["components"])

properties_list

[{'property_name': 'accounts.account_id',
  'property_type': 'string',
  'property_description': "Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.\n\nThe `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.\n\nIf an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.\n\nLike all Plaid identifiers, the `account_id` is case sensitive.",
  'enum': [],
  'nullable': False},
 {'property_name': 'accounts.balances.available',
  'proper

In [10]:
def render_as_markdown_table(properties_list):
    """
    Convert a list of properties to a markdown table.
    """
    # Header for the markdown table
    header = "| Property Name | Type | Description | Enum | Nullable |\n"
    header += "|---------------|------|-------------|------|----------|\n"

    # Generate rows for the table
    rows = []
    for prop in properties_list:
        enum_values = ", ".join(prop["enum"]) if prop["enum"] else "None"
        row = f"| {prop['property_name']} | {prop['property_type']} | {prop['property_description']} | {enum_values} | {prop['nullable']} |"
        rows.append(row)

    return header + "\n".join(rows)

# Render the properties list as a markdown table
markdown_table = render_as_markdown_table(properties_list)

# Display the first few lines of the markdown table for verification
markdown_table.split("\n")

['| Property Name | Type | Description | Enum | Nullable |',
 '|---------------|------|-------------|------|----------|',
 "| accounts.account_id | string | Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.",
 '',
 'The `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.',
 '',
 'If an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.',
 '',
 'Like all Plaid identifiers, the `account_id` is case sensitive. | None | False |',
 

In [45]:
from IPython.display import display
import pandas as pd
def display_as_table(properties_list):
    """
    Display a list of properties as a table in a Jupyter notebook cell.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Display the DataFrame as a table in the Jupyter notebook cell
    display(df)

# Display the properties list as a table in the Jupyter notebook cell
display_as_table(properties_list)

,property_name,property_type,property_description,enum,nullable
0,accounts.account_id,string,Plaid’s unique identifier for the account. Thi...,[],False
1,accounts.balances.available,number,The amount of funds available to be withdrawn ...,[],True
2,accounts.balances.current,number,The total amount of funds in or owed by the ac...,[],True
3,accounts.balances.limit,number,"For `credit`-type accounts, this represents th...",[],True
4,accounts.balances.iso_currency_code,string,The ISO-4217 currency code of the balance. Alw...,[],True
...,...,...,...,...,...
77,item.products,string,A list of products that an institution can sup...,"[assets, auth, balance, identity, investments,...",False
78,item.consented_products,string,A list of products that an institution can sup...,"[assets, auth, balance, identity, investments,...",False
79,item.consent_expiration_time,string,The RFC 3339 timestamp after which the consent...,[],True
80,item.update_type,string,Indicates whether an Item requires user intera...,"[background, user_present_required]",False


In [38]:
example_json = {
    "accounts": [
        {
            "account_id": "BxBXxLj1m4HMXBm9WZZmCWVbPjX16EHwv99vp",
            "balances": {
                "available": 110,
                "current": 110,
                "iso_currency_code": "USD",
                "limit": None,
                "unofficial_currency_code": None
            },
            "mask": "0000",
            "name": "Plaid Checking",
            "official_name": "Plaid Gold Standard 0% Interest Checking",
            "subtype": "checking",
            "type": "depository"
        }
    ],
    "transactions": [
        {
            "account_id": "BxBXxLj1m4HMXBm9WZZmCWVbPjX16EHwv99vp",
            "amount": 2307.21,
            "iso_currency_code": "USD",
            "unofficial_currency_code": None,
            "category": [
                "Shops",
                "Computers and Electronics"
            ],
            "category_id": "19013000",
            "check_number": None,
            "date": "2017-01-29",
            "datetime": "2017-01-27T11:00:00Z",
            "authorized_date": "2017-01-27",
            "authorized_datetime": "2017-01-27T10:34:50Z",
            "location": {
                "address": "300 Post St",
                "city": "San Francisco",
                "region": "CA",
                "postal_code": "94108",
                "country": "US",
                "lat": 40.740352,
                "lon": -74.001761,
                "store_number": "1235"
            },
            "name": "Apple Store",
            "merchant_name": "Apple",
            "payment_meta": {
                "by_order_of": None,
                "payee": None,
                "payer": None,
                "payment_method": None,
                "payment_processor": None,
                "ppd_id": None,
                "reason": None,
                "reference_number": None
            },
            "payment_channel": "in store",
            "pending": False,
            "pending_transaction_id": None,
            "personal_finance_category": {
                "primary": "GENERAL_MERCHANDISE",
                "detailed": "GENERAL_MERCHANDISE_ELECTRONICS"
            },
            "account_owner": None,
            "transaction_id": "lPNjeW1nR6CDn5okmGQ6hEpMo4lLNoSrzqDje",
            "transaction_code": None,
            "transaction_type": "place"
        }
    ],
    "item": {
        "available_products": [
            "balance",
            "identity",
            "investments"
        ],
        "billed_products": [
            "assets",
            "auth",
            "liabilities",
            "transactions"
        ],
        "consent_expiration_time": None,
        "error": None,
        "institution_id": "ins_3",
        "item_id": "eVBnVMp7zdTJLkRNr33Rs6zr7KNJqBFL9DrE6",
        "update_type": "background",
        "webhook": "https://www.genericwebhookurl.com/webhook"
    },
    "total_transactions": 1,
    "request_id": "45QSn"
}

def get_example_for_property(example_json, property_path):
    """
    Extracts an example value for a given property path from the example json.
    """
    parts = property_path.split('.')
    try:
        for part in parts:
            if isinstance(example_json, list):
                example_json = example_json[0]
            example_json = example_json[part]
        return example_json
    except (KeyError, TypeError):
        return None

def append_examples_to_properties(properties, example_json):
    """
    Appends an example to each property in the list of properties based on the example json.
    """
    for prop in properties:
        prop["example"] = get_example_for_property(example_json, prop["property_name"])
    return properties

# Add example data to properties
properties_with_examples = append_examples_to_properties(properties_list, example_json)

properties_with_examples

[{'property_name': 'accounts.account_id',
  'property_type': 'string',
  'property_description': "Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.\n\nThe `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.\n\nIf an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.\n\nLike all Plaid identifiers, the `account_id` is case sensitive.",
  'enum': [],
  'nullable': False,
  'example': 'BxBXxLj1m4HMXBm9WZZmCWVbPjX16EHwv99vp'},
 {'pr

In [41]:
def save_to_html(properties_list, filename):
    """
    Render a list of properties as an HTML table and save to a file.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Save the DataFrame as an HTML file
    df.to_html(filename, index=False, escape=False)

# Save the properties list as an HTML file
save_to_html(properties_with_examples, "transactions get properties table with examples.html")


In [43]:
display_as_table(properties_list)

,property_name,property_type,property_description,enum,nullable,example
0,accounts.account_id,string,Plaid’s unique identifier for the account. Thi...,[],False,BxBXxLj1m4HMXBm9WZZmCWVbPjX16EHwv99vp
1,accounts.balances.available,number,The amount of funds available to be withdrawn ...,[],True,110
2,accounts.balances.current,number,The total amount of funds in or owed by the ac...,[],True,110
3,accounts.balances.limit,number,"For `credit`-type accounts, this represents th...",[],True,None
4,accounts.balances.iso_currency_code,string,The ISO-4217 currency code of the balance. Alw...,[],True,USD
...,...,...,...,...,...,...
77,item.products,string,A list of products that an institution can sup...,"[assets, auth, balance, identity, investments,...",False,None
78,item.consented_products,string,A list of products that an institution can sup...,"[assets, auth, balance, identity, investments,...",False,None
79,item.consent_expiration_time,string,The RFC 3339 timestamp after which the consent...,[],True,None
80,item.update_type,string,Indicates whether an Item requires user intera...,"[background, user_present_required]",False,background
